In [1]:
%load_ext autoreload

import os, sys
import networkx as nx

sys.path.insert(0, "../")
sys.path.insert(0, "/home/j/joppich/.local/lib/python3.11/site-packages/")
#otherwise leidenalg is used in an outdated version ...

import leidenalg as la
print("la", la.version)

%autoreload 2
from mikg.kgraph import *
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from collections import defaultdict, Counter

import random
random.seed(42)

import pickle

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
la 0.10.1


In [2]:
kg = KGraph()
kg.load_kgraph("../data/initial_base_graph.out")

In [3]:
kg.get_edge_between_type()

Counter({('gene', 'geneset'): 841157,
         ('gene', 'gene'): 344408,
         ('TF;gene', 'gene'): 95771,
         ('geneset', 'geneset'): 74634,
         ('TF;gene', 'geneset'): 74544,
         ('gene', 'disease'): 43297,
         ('gene', 'geneset;subcellular_location'): 40554,
         ('disease', 'drug'): 35017,
         ('gene', 'TF;gene'): 19994,
         ('TF;gene', 'TF;gene'): 16120,
         ('gene', 'drug'): 7899,
         ('miRNA;ncRNA', 'gene'): 7848,
         ('TF;gene', 'disease'): 5105,
         ('TF;gene', 'miRNA;ncRNA'): 2099,
         ('TF;gene', 'geneset;subcellular_location'): 2043,
         ('miRNA;ncRNA', 'TF;gene'): 1808,
         ('gene', 'disease;geneset'): 1714,
         ('geneset;subcellular_location', 'geneset;subcellular_location'): 721,
         ('gene', 'miRNA;ncRNA'): 651,
         ('geneset;subcellular_location', 'geneset'): 460,
         ('gene;lncRNA;ncRNA', 'geneset'): 444,
         ('TF;gene', 'drug'): 341,
         ('geneset', 'disease;geneset'

In [4]:
exprDF = pd.read_csv("../seq_data/expression_ctzone_mean_df.tsv", sep="\t")

In [5]:
exprDFs = {}

for group in exprDF.group.unique():
    exprDFs[group] = exprDF[exprDF.group == group].copy()

print([x for x in exprDFs])

['Adipocyte_BZ', 'Adipocyte_CTRL', 'Adipocyte_FZ', 'Adipocyte_IZ', 'Adipocyte_RZ', 'Cardiomyocyte_BZ', 'Cardiomyocyte_CTRL', 'Cardiomyocyte_FZ', 'Cardiomyocyte_IZ', 'Cardiomyocyte_RZ', 'Cycling cells_BZ', 'Cycling cells_CTRL', 'Cycling cells_FZ', 'Cycling cells_IZ', 'Cycling cells_RZ', 'Endothelial_BZ', 'Endothelial_CTRL', 'Endothelial_FZ', 'Endothelial_IZ', 'Endothelial_RZ', 'Fibroblast_BZ', 'Fibroblast_CTRL', 'Fibroblast_FZ', 'Fibroblast_IZ', 'Fibroblast_RZ', 'Lymphoid_BZ', 'Lymphoid_CTRL', 'Lymphoid_FZ', 'Lymphoid_IZ', 'Lymphoid_RZ', 'Mast_BZ', 'Mast_CTRL', 'Mast_FZ', 'Mast_IZ', 'Mast_RZ', 'Myeloid_BZ', 'Myeloid_CTRL', 'Myeloid_FZ', 'Myeloid_IZ', 'Myeloid_RZ', 'Neuronal_BZ', 'Neuronal_CTRL', 'Neuronal_FZ', 'Neuronal_IZ', 'Neuronal_RZ', 'Pericyte_BZ', 'Pericyte_CTRL', 'Pericyte_FZ', 'Pericyte_IZ', 'Pericyte_RZ', 'vSMCs_BZ', 'vSMCs_CTRL', 'vSMCs_FZ', 'vSMCs_IZ', 'vSMCs_RZ']


In [6]:
exprKGs = {}

gsa = GenesetAnnotator()
mNS = MeanNetworkScorer()

for group in exprDFs:
    
    print(group)
    
    gKG = KGraph()
    
    gKG.load_kgraph("../data/initial_base_graph.out")

    gKG.add_gene_expression(exprDFs[group])
    mNS.score_nodes_from_properties(gKG)

    mNS.score(gKG)

    gsa.annotate_genesets(gKG, settype="disease", targettype="gene")
    gsa.annotate_genesets(gKG, settype="geneset", targettype="gene")
    gsa.annotate_genesets(gKG, settype="ncRNA", targettype="gene")
    gsa.annotate_genesets(gKG, settype="drug", targettype=["gene", "disease"])
            
    exprKGs[group] = gKG

Adipocyte_BZ
Measured Genes 11776
Found Genes 10504


/mnt/raidbio/extproj/projekte/regulatory_networks/scripts/../mikg/kgraph.py:1589: RuntimeWarning: invalid value encountered in scalar divide
  edge_zscore = (edge_score-etype2mean[etype])/etype2std[etype]


Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 target nodes for settype=drug and targettype=['gene', 'disease']
Adipocyte_CTRL
Measured Genes 11534
Found Genes 10271
Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 target nodes for settype=drug and targettype=['gene', 'disease']
Adipocyte_FZ
Measured Genes 19844
Found Genes 15856
Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 ta

In [7]:
print(len(exprKGs))
import pickle

with open("zone_exprkgs.pickle", 'wb') as f:
    pickle.dump(exprKGs, f)

55
